In [1]:
import os
import h5py
import numpy as np
import cv2

In [ ]:
#   CARGA DE HDF5
# ==============================
input_h5 = os.path.join("data", "Galaxy10_DECals_NoDuplicated.h5") #ruta del archivo .h5 de origen.
output_dir = "reduced_h5" #carpeta donde se guardarán los HDF5 reducidos.
os.makedirs(output_dir, exist_ok=True)

IMG_SIZE = 128

h5 = h5py.File(input_h5, "r")
images = h5["images"]
labels = h5["ans"] #obtiene el dataset de etiquetas, cuyo nombre en el HDF5 original es ans.
N = images.shape[0]
print("Total imágenes:", N)

Total imágenes: 17675


In [ ]:
#   SPLIT train/val/test
# ==============================
indices = np.arange(N)
np.random.seed(42)
np.random.shuffle(indices)

train_end = int(0.7 * N) #calculan límites para 70% train, 15% val, 15% test.
val_end   = int(0.85 * N)

splits = {
    "train": indices[:train_end],
    "val":   indices[train_end:val_end],
    "test":  indices[val_end:]
}

print(f"splits[train]: {len(splits['train'])}, splits['Val']: {len(splits['val'])}, splits['Test']: {len(splits['test'])}")

splits[train]: 12372, splits['Val']: 2651, splits['Test']: 2652


In [10]:
# FUNCIÓN DE REDIMENSIONADO
# =============================
def resize_img(img):
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

In [ ]:
# =============================
# CREAR LOS NUEVOS ARCHIVOS H5
# =============================
for split_name, split_idx in splits.items():
    out_path = os.path.join(output_dir, f"{split_name}_{IMG_SIZE}.h5") #Para cada split (train, val, test) se crea out_path

    file_out = h5py.File(out_path, "w")
    d_imgs = file_out.create_dataset(
        "images", 
        (len(split_idx), IMG_SIZE, IMG_SIZE, 3),
        dtype=np.uint8  #crea el dataset con imagenes en 128x128 y type uint8
    )
    d_lbls = file_out.create_dataset(
        "labels", 
        (len(split_idx),),
        dtype=np.int32  #crea el dataset de etiquetas.
    )

    print(f"Generando {split_name}: {len(split_idx)} elementos...")

    for i, orig_i in enumerate(split_idx):
        img = images[orig_i]              # sin cargar a RAM
        img_resized = resize_img(img)     # reduce tamaño
        d_imgs[i] = img_resized
        d_lbls[i] = labels[orig_i]

        if i % 2000 == 0: #imprimimos progreso cada 2000 imagenes procesadas
            print(f"{i}/{len(split_idx)} procesadas...")

    file_out.close()
    print(f"{split_name} guardado en: {out_path}")

h5.close()
print("\nProceso finalizado.")

Generando train: 12372 elementos...
0/12372 procesadas...
2000/12372 procesadas...
4000/12372 procesadas...
6000/12372 procesadas...
8000/12372 procesadas...
10000/12372 procesadas...
12000/12372 procesadas...
train guardado en: reduced_h5\train_128.h5
Generando val: 2651 elementos...
0/2651 procesadas...
2000/2651 procesadas...
val guardado en: reduced_h5\val_128.h5
Generando test: 2652 elementos...
0/2652 procesadas...
2000/2652 procesadas...
test guardado en: reduced_h5\test_128.h5

Proceso finalizado.
